In [1]:
import pandas as pd
import numpy as np
from find_path import find_file
import sqlalchemy
import db_utils
import warnings 
warnings.filterwarnings('ignore')

# Matching with fragments predicted by the DMI predictor

In [3]:
# load mutations
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query = '''select * from milena_db.DDD_and_clinvar_mutations_2023_release as df'''
mut_data = pd.read_sql_query(query, con=eng)
mut_data_sub1 = mut_data[mut_data['allele_id'].isna()] #the data types were floats for some reason and mixed with NaN values
mut_data_sub2 = mut_data[~(mut_data['allele_id'].isna())]
mut_data_sub2['allele_id'] = mut_data_sub2['allele_id'].astype('int')
mut_data_sub2['variation_id'] = mut_data_sub2['variation_id'].astype('int')
mut_data_sub1['allele_id'] = None
mut_data_sub1['variation_id'] = None
mut_data = pd.concat([mut_data_sub2, mut_data_sub1], axis=0)

# load John's table of predicted DMI interfaces
query = pd.read_sql_query("select * from chopyan_db.HuRI_BioPlex_lit17_union as df where DMIMatchScore >= 0.7", con = eng)
dmi = pd.DataFrame(query)

# I saved all the columns I think I will need later:
dmi = dmi[['intx_ID', 'Accession', 'Elm', 'NDDSLiMProtein', 'SLiMProtein', 'SLiMMatch', 'NDDDomainProtein', 'DomainProtein', 'DomainID1', 'DomainName1', 'DomainMatch1', 'DomainMatchSequence1', 'DomainMatchFound1', 'DomainMatchRequired1', 'DomainID2', 'DomainName2', 'DomainMatch2', 'DomainMatchSequence2', 'DomainMatchFound2', 'DomainMatchRequired2', 'DMIMatchScore', 'Source_lit17', 'Source_HuRI', 'Source_BioPlex']]

In [12]:
# making a list of slims:
slim = dmi[['SLiMProtein','Accession','SLiMMatch']]
slim.rename(columns = {'SLiMProtein':'fragment_protein','Accession':'fragment_id', 'SLiMMatch':'fragment_pos'}, inplace = True)
slim['fragment_type'] = 'slim'
slim.drop_duplicates(inplace = True)
slim.reset_index(inplace=True, drop=True)

# making a list of domains 1; like slims, they are always provided (in each row)
dom_1 = dmi[['DomainProtein','DomainID1','DomainMatch1']]
dom_1.rename(columns = {'DomainProtein':'fragment_protein','DomainID1':'fragment_id', 'DomainMatch1':'fragment_pos'}, inplace = True)
dom_1['fragment_type'] = 'domain'
dom_1.drop_duplicates(inplace=True)
dom_1.reset_index(inplace=True, drop=True)

# second domain
dom_2 = dmi[~(dmi['DomainID2'].isna()) & ~(dmi['DomainMatch2'].isna())][['DomainProtein','DomainID2','DomainMatch2']]
dom_2.rename(columns={'DomainProtein':'fragment_protein','DomainID2':'fragment_id','DomainMatch2':'fragment_pos'}, inplace = True)
dom_2['fragment_type'] = 'domain'
dom_2.drop_duplicates(inplace=True)
dom_2.reset_index(inplace=True, drop=True)

# concatenate them:
frag = pd.concat([slim,dom_1,dom_2], axis=0, ignore_index=True)
frag

,fragment_protein,fragment_id,fragment_pos,fragment_type
0,A0A087WUL8,ELME000443,258-264,slim
1,A0A087WUL8,ELME000084,211-214,slim
2,A0A087WUL8,ELME000045,126-132,slim
3,A0A096LP49,ELME000271,246-252,slim
4,A0A096LP49,ELME000388,250-254,slim
...,...,...,...,...
25032,Q05655,SM00133,604-667,domain
25033,Q9NRM7,SM00133,974-1044,domain
25034,Q08999,PF01858,417-609,domain
25035,Q9Y5S2,SM00133,343-405,domain


In [17]:
# merging frag table with the mut_data on uniprot_id

# # to check if a mutation falls within a certain fragment in a certain protein, we need to find fragments of the protein where the mutation lies
overlap = frag.merge(mut_data, how='inner', left_on='fragment_protein', right_on='uniprot_id')
# I filtered clinvar additionally, so I might not have the same numbers as Kristina

# now let's look for overlap:
overlap['fragment_pos_for_overlap'] = [''.join([a for a in i.split('|')]) if len([a for a in i.split('|')]) == 1 else [a for a in i.split('|')] for i in overlap['fragment_pos'].values]

overlap_list = overlap[overlap['fragment_pos_for_overlap'].apply(lambda x: type(x) == list)] # dataset where there is only 1 predicted position per fragment
overlap_not_list = overlap[~(overlap['fragment_pos_for_overlap'].apply(lambda x: type(x) == list))]
overlap_list.reset_index(inplace=True, drop=True)
overlap_not_list.reset_index(inplace=True, drop=True)

overlap_not_list['overlap'] = [1 if (a >= int(b.split('-')[0])) and (a <= int(b.split('-')[1])) else 0 for a, b in zip(overlap_not_list['aa_pos'], overlap_not_list['fragment_pos_for_overlap'])]
overlap_list['overlap'] = [1 if 1 in [1 if (a >= int(x.split('-')[0])) and (a <= int(x.split('-')[1])) else 0 for x in b] else 0 for a, b in zip(overlap_list['aa_pos'], overlap_list['fragment_pos_for_overlap'])]

overlap = pd.concat([overlap_list, overlap_not_list], axis=0)
overlap.reset_index(inplace=True, drop=True)

# annotate proteins for NDDs
query = '''select * from milena_db.NDD_genes_list_v3 as df'''
ndd = pd.read_sql_query(query, con=eng)
ndd = ndd['uniprot_id'].unique().tolist()
overlap['ndd_protein'] = [1 if i in ndd else 0 for i in overlap['uniprot_id']]

# fix dtype in chromosome column:
overlap['chromosome'] = [int(i) if i.isdigit() else i for i in overlap['chromosome']]

# select columns and filter for those rows where overlap == 1
overlap = overlap[['fragment_type','fragment_protein','fragment_id','fragment_pos','allele_id','variation_id','chromosome', 'composite_id', 'aa_change', 'pathogenicity', 'ndd_phe', 'ndd_protein', 'overlap']]
overlap = overlap[overlap['overlap'] == 1]
overlap.drop(columns='overlap', inplace=True)
overlap.drop_duplicates(inplace=True)
overlap.reset_index(inplace=True, drop=True)

overlap

,fragment_type,fragment_protein,fragment_id,fragment_pos,allele_id,variation_id,chromosome,composite_id,aa_change,pathogenicity,ndd_phe,ndd_protein
0,domain,Q92625,SM00248,79-108|112-141|148-177|181-210|214-243|246-275,2219276,2228534,6,6_34981954_C_A,Leu234Ile,uncertain,0,0
1,domain,Q92625,SM00248,79-108|112-141|148-177|181-210|214-243|246-275,2704296,2537954,6,6_34981799_T_G,Phe182Cys,uncertain,0,0
2,domain,Q92625,SM00248,79-108|112-141|148-177|181-210|214-243|246-275,2762399,2593541,6,6_34970074_T_C,Tyr115His,uncertain,0,0
3,domain,A1X283,SM00326,155-210|224-279|371-426|853-911,303282,352815,5,5_172346198_T_C,Ile376Val,uncertain,0,0
4,domain,A1X283,SM00326,155-210|224-279|371-426|853-911,443751,451250,5,5_172338487_C_G,Gly873Ala,uncertain,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
49921,domain,Q9Y587,PF01217,1-141,2383119,2384873,14,14_31069903_A_G,Ile67Val,uncertain,0,0
49922,domain,Q9Y587,PF01217,1-141,2404999,2431495,14,14_31066240_G_A,Arg15Gln,uncertain,1,0
49923,domain,Q9Y587,PF01217,1-141,2411751,2439139,14,14_31069856_A_G,Glu51Gly,uncertain,1,0
49924,domain,Q9Y587,PF01217,1-141,2448610,2469842,14,14_31069883_G_A,Arg60Gln,uncertain,0,0


# Push to mysql

In [5]:
#check for NaN values:
col_list = overlap.columns.tolist()
for col in col_list:
    if ((overlap[col].isna()).any()).any():
        print(col) 
#none of the columns contain NaN values

max([len((str(i))) for i in overlap['pathogenicity']])

allele_id
variation_id


11

A note about the primary key: protein, fragment_id and fragment_pos is what uniquely identifies a fragment, and on the other hand composite id and aa_change is what uniquely defines a mutation (aa_change also needs to be there because a gene can code for 2 proteins and then aa_change can be different, even though the genomic location of the mutation is the same), so the primary key combines all these columns.
Theoretically, composite_id and aa_change could be replaced by allele_id, variation_id and chromosome, but the problem is that DDD mutations don't have this, and there are None values there, so technically there are duplicates for these columns.

In [6]:
#  CREATE TABLE `DMI_overlap_ddd_and_clinvar_2023_release` (
#    `fragment_type` text(7) NOT NULL,
#    `fragment_protein` varchar(12) NOT NULL,
#    `fragment_id` varchar(12) NOT NULL,
#    `fragment_pos` varchar(130) NOT NULL,
#    `allele_id` int(8) DEFAULT NULL,
#    `variation_id` int(8) DEFAULT NULL,
#    `chromosome` varchar(2) NOT NULL,
#    `composite_id` varchar(18) NOT NULL,
#    `aa_change` varchar(12) NOT NULL,
#    `pathogenicity` text(12) NOT NULL,
#    `ndd_phe` int(1) NOT NULL,
#    `ndd_protein` int(1) NOT NULL,
#    PRIMARY KEY (`fragment_protein`,`fragment_id`,`fragment_pos`,`composite_id`,`aa_change`)
#  ) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4;

In [17]:
connect = db_utils.get_connection()
cursor = connect.cursor()

db_table = 'milena_db.DMI_overlap_ddd_and_clinvar_2023_release'
query = """insert into %s 
        (fragment_type, fragment_protein, fragment_id, fragment_pos, allele_id, variation_id, chromosome, composite_id, aa_change, pathogenicity, ndd_phe, ndd_protein)
        values (%%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s)""" % (db_table)

for i in range(overlap.shape[0]):
    cursor.execute(query, tuple(overlap.loc[i]))